In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('deals.xlsx')
df = df.dropna()

In [3]:
df.columns

Index([u'цена', u'a', u'b', u'условие', u'pl сделки', u'метка для обучения'], dtype='object')

In [4]:
df = df[[ u'a', u'b', u'pl сделки', u'метка для обучения']]
df.columns = ['a','b','pl','y']
df['y'] = df['y'].astype('int')

In [5]:
df

,a,b,pl,y
0,0.0,0.0,0,0
1,-740.0,547600.0,0,0
2,-2500.0,6250000.0,0,0
3,-3290.0,10824100.0,0,0
4,2030.0,4120900.0,2300,1
5,2300.0,5290000.0,4650,1
6,4650.0,21622500.0,1540,1
7,1540.0,2371600.0,1640,1
8,1640.0,2689600.0,-5550,0
9,-5550.0,30802500.0,0,0


Допустим, что y = 0 - неудачная сделка, даже если она не принесла убытков.

Теперь сделаем классификатор.

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [7]:
X,y = df[['a','b']], df['y']

In [8]:
clf = LogisticRegression()
scaler = StandardScaler()
pipeline = Pipeline([('scaler',scaler),('Logit',clf)])

Дата сет очень маленьки, поэтому делать train test бессмысленно. Просто посмотрим по кроссвалидации какую долю правильных ответов выдает классификатор.

In [9]:
cross_val_score(pipeline,X,y,cv=3)

array([ 0.71428571,  0.83333333,  0.83333333])

Ну будем считать, что работает. Теперь обучем на всех данных.

In [10]:
pipeline.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('Logit', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

Посмотрим, что он предсказал. Выведем и вероятность и предсказанный класс с порогом 1.

In [11]:
df['y_proba'] = pipeline.predict_proba(X)[:,1]
df['y_pred_05'] = (pipeline.predict_proba(X)[:,1]>0.5).astype('int')
df

,a,b,pl,y,y_proba,y_pred_05
0,0.0,0.0,0,0,0.167445,0
1,-740.0,547600.0,0,0,0.122703,0
2,-2500.0,6250000.0,0,0,0.078218,0
3,-3290.0,10824100.0,0,0,0.074166,0
4,2030.0,4120900.0,2300,1,0.464563,0
5,2300.0,5290000.0,4650,1,0.525939,1
6,4650.0,21622500.0,1540,1,0.940089,1
7,1540.0,2371600.0,1640,1,0.364172,0
8,1640.0,2689600.0,-5550,0,0.383251,0
9,-5550.0,30802500.0,0,0,0.107305,0


Видно, что классификатор неправильно определил два трейда - 4 и 7. Они на самом деле положительные.

Чтобы быстро это понимать существует матрица ошибок.

In [12]:
pd.options.display.max_colwidth = 100
help_table = pd.DataFrame([[[u'True positive (ТP)',u'Положительное срабатывание'],
                            [u'False positive (FP)',u'Ложное срабатыватение','Ошибка II']],
                           [[u'False negative (FN)',u'Ложный пропуск','Ошибка I'],
                            [u'True negative (TN)',u'Верный пропуск']]],
                          index=[u'Предсказанный положительный',
                                 u'Предсказанный отрицательный'],
                          columns=[u'Реально положительный',
                                   u'Реально отрицательный'])
help_table = help_table.T.ix[::-1]
help_table[help_table.columns[::-1]]

,Предсказанный отрицательный,Предсказанный положительный
Реально отрицательный,"[True negative (TN), Верный пропуск]","[False positive (FP), Ложное срабатыватение, Ошибка II]"
Реально положительный,"[False negative (FN), Ложный пропуск, Ошибка I]","[True positive (ТP), Положительное срабатывание]"


В sklearn это все автоматизировано. Посмотрим на матрицу ошибок и classification report. Важно понимать, что порог сейчас равен 0.5. Так же выведу сколько всего 0 и 1.

In [13]:
def full_report(y_true, y_pred):
    print 'Class count'
    print '-'*79
    print df.groupby('y')['a'].count()
    print '-'*79
    print 'Confusion matrix'
    print '-'*79
    print confusion_matrix(y_true, y_pred)
    print '-'*79
    print 'Classification report'
    print '-'*79
    print classification_report(y_true, y_pred)
    
full_report(y,pipeline.predict(X))

Class count
-------------------------------------------------------------------------------
y
0    15
1     4
Name: a, dtype: int64
-------------------------------------------------------------------------------
Confusion matrix
-------------------------------------------------------------------------------
[[15  0]
 [ 2  2]]
-------------------------------------------------------------------------------
Classification report
-------------------------------------------------------------------------------
             precision    recall  f1-score   support

          0       0.88      1.00      0.94        15
          1       1.00      0.50      0.67         4

avg / total       0.91      0.89      0.88        19



Что мы видим? В confusion matrix видно, что классификатор неправильно определил два объекта 1 класса в класс ноль. В classification report таже инфа, только в процентах. Например в первой строчке 0.88 означает, что из всех объектов нулевого класса определенных классификатором только 88% реально из нулевого класса. Действительно - классификатор отнес к 0 17 объектов, а их всего 15.

In [14]:
15/17.

0.8823529411764706

Что дальше? Давай увеличим порог, и посмотрим, что получится. Возьмем порог равный 0.4, т.к. мы хочется выделить больше объектов 1 класса.

In [15]:
y_pred = (pipeline.predict_proba(X)[:,1] > 0.4).astype('int')

In [16]:
full_report(y,y_pred)

Class count
-------------------------------------------------------------------------------
y
0    15
1     4
Name: a, dtype: int64
-------------------------------------------------------------------------------
Confusion matrix
-------------------------------------------------------------------------------
[[15  0]
 [ 1  3]]
-------------------------------------------------------------------------------
Classification report
-------------------------------------------------------------------------------
             precision    recall  f1-score   support

          0       0.94      1.00      0.97        15
          1       1.00      0.75      0.86         4

avg / total       0.95      0.95      0.94        19



О чудо! Показатель классификатора улучшились! Теперь мы неправильно определили только один объек 1 класса.

Что теперь сказать о прибыли? Как ее максимизировать? Посмотрим, сколько в среднем мы теряем на неудачном трейде и зарабатываем на удачном.

In [17]:
df.groupby('y')['pl'].mean()

y
0    -444.0
1    2532.5
Name: pl, dtype: float64

Получается, что ложный пропуск обходиться гораздо дороже, чем ложное срабатывание (обычно все наоборот, например в кредитном скоринге). Теперь меня порог и перемножая confusion matrix на цену ошибки можно максимизировать прибль.

In [18]:
cost_matrix = np.array([[0,  -444.0],
                   [0,  2532.5]])

In [19]:
thrs = np.linspace(0,1,21)
pl = np.zeros(len(thrs))

for i, thr in enumerate(thrs):
    y_pred = (pipeline.predict_proba(X)[:,1] > thr).astype('int')
    conf_m = confusion_matrix(y,y_pred)
    pl[i] = np.sum(conf_m * cost_matrix)
    
pls = pd.DataFrame([thrs,pl]).T

In [20]:
pls

,0,1
0,0.00,3470.0
1,0.05,3470.0
2,0.10,4802.0
3,0.15,6578.0
4,0.20,7910.0
5,0.25,7910.0
6,0.30,9242.0
7,0.35,9242.0
8,0.40,7597.5
9,0.45,7597.5


И ты сразу видишь, что достигаешь максимума прибыли при пороге 0.3 или 0.35.

In [21]:
pls[pls[1] == pls[1].max()]

,0,1
6,0.30,9242.0
7,0.35,9242.0


Такая вот история).